In [1]:
# Load custom VAEDCGAN model
from celeb_gan_builder import *

class Base_Transform():
    def __init__(self, configuration=None):
        self.configuration = configuration

    def t_transform(self):
        return transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=self.configuration['mean'], std=self.configuration['std'])])


    def ut_transform(self):

        mean, std = self.configuration['mean'], self.configuration['std']
        umean, ustd = [mean[0], mean[1], mean[2]], [std[0], std[1], std[2]]

        for i in range(3):
            ustd[i] = 1.0/ustd[i]
            umean[i] = -umean[i]*ustd[i]

        t = transforms.Compose([transforms.Normalize(mean=umean, std=ustd)])

        return t

class DataBuilder(data.Dataset):

        def __init__(self, configuration):

                self.configuration = configuration
                if "shuffle" in configuration['sampler'].keys():
                    self.loader = data.DataLoader(dataset=self, batch_size=configuration['sampler']['bs'], shuffle=configuration['sampler']['shuffle'], num_workers=12)
                else:
                    self.loader = data.DataLoader(dataset=self, batch_size=configuration['sampler']['bs'], sampler=configuration['sampler']['sampler'], num_workers=12)

                self.ut = Base_Transform(bt_configuration_face).ut_transform()

        def __getitem__(self, index):
                return self.configuration['decoder'](self.configuration['data'][index], self.configuration['transform'])

        def __len__(self):
                return len(self.configuration['data'])

configurations = {'train': {'data':[], 'decoder':None, 'length':-1, 'sampler':None, 'transform':None}, 'val': {'data':[], 'decoder':None, 'length':-1, 'sampler':None, 'transform':None}
                  ,'test': {'data':[], 'decoder':None, 'length':-1, 'sampler':None, 'transform':None}}

datapath ={'train':"/scratch/Jack/datasets/True_Relations_Dataset/train_test_eval_splits/annotator_consistency3(used in our paper)/pairwise_face_train_16.txt",
'val':"/scratch/Jack/datasets/True_Relations_Dataset/train_test_eval_splits/annotator_consistency3(used in our paper)/pairwise_face_eval_16.txt",
'test':"/scratch/Jack/datasets/True_Relations_Dataset/train_test_eval_splits/annotator_consistency3(used in our paper)/pairwise_face_test_16.txt"}

# No cropping or flipping initially

proper_size = (64,64)
bt_configuration_face={'mean':[0.506 ,0.426,0.383],'std':[0.304 ,0.283 ,0.283]}
bt = [Base_Transform(bt_configuration_face).t_transform()]
configurations['train']['transform'] = bt
configurations['test']['transform'] = bt
configurations['val']['transform'] = bt

##### Sampler Init

weight_per_class = [0.] * 16
classes = [x for x in range(16)]
class_numbers = {'train':{x: 0 for x in classes},'val':{x: 0 for x in classes}, 'test':{x: 0 for x in classes}}

######

for face in glob.glob("/scratch/Jack/datasets/CelebA/img_resized_celeba/celebA/*.jpg"):
    datum = {"face": face}
    configurations['train']['data'].append(datum)

# Decode from filepaths to data
def mapping_decoder(datum, t):

    image_file_face = datum["face"]
    face_img = t[0](PIL.Image.open(image_file_face).convert("RGB"))
    sample = {"Face": face_img}

    return sample

############# Subsets

configurations['train']['subset_percent'] = 1 #
configurations['val']['subset_percent'] = 1 #
configurations['test']['subset_percent'] = 1 #

def random_subset_selection():
    for subset in ['train','val','test']:
        index = 0
        count_max = len(configurations[subset]['data'])
        for i in range(count_max):
            if random.random() >= configurations[subset]['subset_percent']:
                datum = configurations[subset]['data'].pop(index)
                if subset == "train":
                    label = datum["relationship"]
                    class_numbers[subset][int(label)] -= 1
            else:
                index += 1

def specific_subset_selection():
    for subset in ['train','val','test']:
        index = 0
        count_max = len(configurations[subset]['data'])
        for i in range(count_max):
            if i % int(1.0/configurations[subset]['subset_percent']) != 0:
                datum = configurations[subset]['data'].pop(index)
            else:
                index += 1

specific_subset_selection()

############# Set up configurations for sampling and decoding

configurations['train']['decoder'] = mapping_decoder
configurations['train']['sampler'] = {'bs':128,'shuffle':True}
configurations['val']['decoder'] = mapping_decoder
configurations['val']['sampler'] = {'bs':128,'shuffle':False}
configurations['test']['decoder'] = mapping_decoder
configurations['test']['sampler'] = {'bs':128,'shuffle':False}

####

db_train = DataBuilder(configurations['train'])
db_test = DataBuilder(configurations['test'])
db_val = DataBuilder(configurations['val'])

V = GAN_Building(model_choice="DCGAN", dbs={'train':db_train, 'val':db_val, 'test':db_test}, result_path="/scratch/Jack/research lab/True_Relations/", proper_size=proper_size)

In [2]:
V.scheduler['start'] = 10
V.directory_models = V.directory + "/models/"
V.GAN_encoder.load_state_dict(torch.load(V.directory_models + "GAN_enc_" + str(V.scheduler['start'] - 1) + ".model"))
V.GAN_decoder.load_state_dict(torch.load(V.directory_models + "GAN_dec_" + str(V.scheduler['start'] - 1) + ".model"))
V.GAN_discriminator.load_state_dict(torch.load(V.directory_models + "GAN_dsc_" + str(V.scheduler['start'] - 1) + ".model"))

In [3]:
noise = [Variable(torch.randn((1, V.GAN_generator.z)).view(-1, V.GAN_generator.z, 1, 1)).cuda()]
#V.GAN_generator.decoder(noise[0])

In [28]:
from bokeh.io import  push_notebook, show, output_notebook, output_file
from bokeh.layouts import column, layout, row, widgetbox
from bokeh.plotting import figure, curdoc
from bokeh.models import CustomJS
from bokeh.events import ButtonClick
from bokeh.models.widgets import Tabs, Panel, Slider, Button, TextInput


In [5]:
button_callback = CustomJS(code="""
if (IPython.notebook.kernel !== undefined) {
    var kernel = IPython.notebook.kernel;
    cmd = "update_contextual_sample(0,0,random)";
    kernel.execute(cmd, {}, {});}
""")

callback = CustomJS(code="""
    if (IPython.notebook.kernel !== undefined) {
    var kernel = IPython.notebook.kernel;
    cmd = "update_contextual_sample('"+cb_obj.title+"','"+cb_obj.value+"', 'notrandom')";
    kernel.execute(cmd, {}, {});}
""")

z_dim = 99
z_groups = 3

z_noise = [0 for k in range(z_dim)]
#z_interactive = [Slider(width=100, height=2,start=-3.0, end=3.0, value=0.0, step=0.01, title="", callback=callback, callback_policy="mouseup") for k in range(99)]
#for i in range(99):
#    z_interactive[i].height = 0
#    z_interactive[i].sizing_mode = 'scale_both'
#button = Button(label="Random sampling")
#button.js_on_event(ButtonClick, button_callback)

#w_tab = Panel(child=row(column(z_interactive), button, column(column(noise_samples[0])) ), title="z sampling")

# must give a vector of image data for image parameter


In [78]:

#bt_configuration_face={'mean':[0.506 ,0.426,0.383,0],'std':[0.304 ,0.283 ,0.283,1]}
bt_configuration_face={'mean':[0.5 ,0.5,0.5,0],'std':[0.5 ,0.5 ,0.5,1]}
def ut_transform():
    global bt_configuration_face
    mean, std = bt_configuration_face['mean'], bt_configuration_face['std']
    umean, ustd = [mean[0], mean[1], mean[2]], [std[0], std[1], std[2]]

    for i in range(3):
        ustd[i] = 1.0/ustd[i]
        umean[i] = -umean[i]*ustd[i]

    t = transforms.Compose([transforms.Normalize(mean=umean, std=ustd)])

    return t
def decode_image(image):
    random_sample = image.cpu().data.numpy()[0]
    random_sample = random_sample.swapaxes(0,2)
    random_sample = np.concatenate((random_sample,np.multiply(np.ones((64,64,1)),1)), axis=2)
    ut = ut_transform()
    intt = torch.FloatTensor(random_sample)
    intt = intt.permute(2,1,0)
    innt = torch.unsqueeze(intt,0)
    t = ut(intt)
    t = t.clamp( 0, 1)
    t = t.numpy().swapaxes(0,2)
    t = t[:,::-1,:]
    t = t.swapaxes(0,1)
    t = np.multiply(t, 255).astype(np.uint8)
    return t
def update_image():
    random_sample = V.GAN_decoder.decoder(z_sample_noise[0])
    my_image.image_rgba(image=[decode_image(random_sample)], x=0, y=0, dw=16, dh=16)
    
    push_notebook(handle=bokeh_handle)
    
button_callback = CustomJS(code="""
if (IPython.notebook.kernel !== undefined) {
    var kernel = IPython.notebook.kernel;
    cmd = "update_image()";
    kernel.execute(cmd, {}, {});}
""")

callback = CustomJS(code="""
if (IPython.notebook.kernel !== undefined) {
var kernel = IPython.notebook.kernel;
cmd = "update_contextual_sample('"+cb_obj.title+"','"+cb_obj.value+"')";
kernel.execute(cmd, {}, {});}
""")
def update_contextual_sample(name, value):
    global z_sample_noise, z_interactive
    for k in range(z_dim):
        if z_interactive[k].title == name:
            z_interactive[k].value = float(value)
            z_sample_noise[0][0].data[k] = float(value)
    
#img.append(figure(plot_width=128*2, plot_height=12*2, x_range=[0, 128], y_range=[128, 0],  x_axis_location="above",  title=""))
my_image = figure(x_range=(0, 16), y_range=(0, 16))

z_sample_noise = [Variable(torch.zeros((1, V.GAN_generator.z)).view(-1, V.GAN_generator.z, 1, 1)).cuda()]
random_sample = V.GAN_decoder.decoder(z_sample_noise[0])

my_image.image_rgba(image=[decode_image(random_sample)], x=0, y=0, dw=16, dh=16)
my_image.height = 128
my_image.width = 128
my_image.axis.visible = False
my_image.toolbar_location = None 
button = Button(label="Random sampling")
button.js_on_event(ButtonClick, button_callback)   

z_interactive = [Slider(start=-3.0, end=3.0, value=0.0, step=0.01, height=0, title="z_{}".format(k), callback=callback, callback_policy="mouseup") for k in range(z_dim)]

w_tabs = Panel(child=row(my_image, column(z_interactive), button), title="z sampling")
tabss = Tabs(tabs=[w_tabs])
bokeh_handle = show(tabss, notebook_handle=True)

In [77]:
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.colors import HSL, Color
from bokeh.util.hex import axial_to_cartesian
from bokeh.models import HoverTool
from bokeh import events
from bokeh.util.hex import cartesian_to_axial

output_notebook()

class z_node():
    def __init__(self, q, r, name):
        self.value = 0
        self.q = q
        self.r = r
        self.name = name
        
    def __str__(self):
        return "[{},{}] {}={}".format(self.q, self.r, self.name, self.value)
        
z = []

for i in range(-int(math.sqrt(z_dim)),int(math.sqrt(z_dim))):
    for j in range(-int(math.sqrt(z_dim)),int(math.sqrt(z_dim))):
        z.append(z_node(i,j,"z_{}".format(0)))

def hex_distance(a):
    return (abs(a.q) 
          + abs(a.q + a.r)
          + abs(a.r)) / 2
        
z = sorted(z, key=lambda z_node:hex_distance(z_node))
z_len = 99
z = z[0:z_len]

for k in range(len(z)):
    z[k].name = "z_{}".format(k)
    z[k].value = 0

q = None
r = None
v_list = None
color_vals = None
p = figure(plot_width=100, plot_height=400, tools="")
p.grid.visible = False
p.axis.visible = False

def init_hex():
    global q, r, v_list, color_vals, p
    q_list = []
    r_list = []
    v_list = []
    color_vals = []
    for k in range(len(z)):
        q_list.append(z[k].q)
        r_list.append(z[k].r)
        v_list.append(z[k].value)
        intense = (3+random.randint(-3,3))/6
        color_vals.append(HSL(intense*40,1,0.5-0.06*hex_distance(z[k])))
        
    q = np.array(q_list)
    r = np.array(r_list)
    cv = np.array(color_vals)
    
    p.background_fill_color  = "black"
    p.height = 256
    p.width = 256
    p.hex_tile(q, r, size=1, fill_color=color_vals,line_color="black", alpha=0.5, hover_color="yellow", name="g")

#init_hex()

x = 0
y = 0
KLD = 0
def change_event(event_string):
    global x, y, z, color_vals
    x,y,_,_ = event_string.split(",")
    x = x.split("=")[1]
    x = float(x)
    y = y.split("=")[1]
    y = float(y)
    x, y = cartesian_to_axial(x, y, 1, "pointytop")
    
    for k in range(len(z)):
        if z[k].q == x and z[k].r == y:
            z[k].value += 1.0
            intense = (3+z[k].value)/6
            color_vals[k]= HSL(intense*40,1,0.5-0.06*hex_distance(z[k]))
    
    push_notebook(handle=bokeh_handle)
    
z = np.random.rand(99,1).astype(np.float64)
p.image(image=[z], x=0, y=0, dw=16,dh=16)
def text_update(update_string):
    # Update a numpy array with rgba values of z[k]
    global p
    p.image(image=z)
    push_notebook(handle=bokeh_handle)
    
    
def callback_event():
    return CustomJS(code="""
    if (IPython.notebook.kernel !== undefined) {
        var kernel = IPython.notebook.kernel;
        cmd = "change_event()";
        kernel.execute(cmd, {}, {});}
    """)
def text_callback():
    return CustomJS(code="""
    var f = cb_obj['value']
    if (IPython.notebook.kernel !== undefined) {
        var kernel = IPython.notebook.kernel;
        cmd = "text_update('"+f+"')";
        kernel.execute(cmd, {}, {});}
    """)

def display_event(attributes=[], style = 'float:left;clear:left;font_size=0.5pt'):
    "Build a suitable CustomJS to display the current event in the div model."
    
    return CustomJS(code="""
        var attrs = %s; var args = [];
        for (var i=0; i<attrs.length; i++ ) {
            args.push(attrs[i] + '=' + Number(cb_obj[attrs[i]]).toFixed(2));
        }
        line = args.join(", ")
        if (IPython.notebook.kernel !== undefined) {
        var kernel = IPython.notebook.kernel;
        cmd = "change_event('"+line+"')";
        kernel.execute(cmd, {}, {});}
        
    """ % (attributes))

point_attributes = ['x','y','sx','sy']  
point_events =  [events.Tap, events.DoubleTap, events.Press]
for event in range(len(point_events)):
    p.js_on_event(point_events[event],display_event(point_attributes))
    
hover = HoverTool(tooltips=None)
p.add_tools(hover)
t = TextInput(callback = text_callback())
tab = Panel(child=row(p,t), title="z sampling")
tabs = Tabs(tabs=[tab])
bokeh_handle = show(tabs, notebook_handle=True)
# To do
# Mouse events for click and double click duration changes z
# after release, z's update rgba image of latent code
# and then you update the example

Loading BokehJS ...

In [56]:
print(z[0].q == x, z[0].r == y)
print(x,y, z[0].value)
print(z[1].q == x, z[1].r == y)
print(x,y, z[1].value)
print(KLD)

True True
0 0 0
False True
0 0 0
fsfs


In [43]:
import numpy as np

from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin

n = 50000
x = np.random.standard_normal(n)
y = np.random.standard_normal(n)

bins = hexbin(x, y, 0.1)

p = figure(title="Manual hex bin for 50000 points", tools="wheel_zoom,reset",
           match_aspect=True, background_fill_color='#440154')
p.grid.visible = False

p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=bins,
           fill_color=linear_cmap('counts', 'Viridis256', 0, max(bins.counts)))

output_file("hex_tile.html")

bins[0] = 3
show(p)

In [64]:

N = 500
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)
print(d.shape, d.dtype)

(500, 500) float64


In [ ]:
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.colors import HSL, Color
from bokeh.util.hex import axial_to_cartesian
from bokeh.models import HoverTool
from bokeh import events
from bokeh.util.hex import cartesian_to_axial

output_notebook()

class z_node():
    def __init__(self, q, r, name):
        self.value = 0
        self.q = q
        self.r = r
        self.name = name
        
    def __str__(self):
        return "[{},{}] {}={}".format(self.q, self.r, self.name, self.value)
        
z = []

for i in range(-int(math.sqrt(z_dim)),int(math.sqrt(z_dim))):
    for j in range(-int(math.sqrt(z_dim)),int(math.sqrt(z_dim))):
        z.append(z_node(i,j,"z_{}".format(0)))

def hex_distance(a):
    return (abs(a.q) 
          + abs(a.q + a.r)
          + abs(a.r)) / 2
        
z = sorted(z, key=lambda z_node:hex_distance(z_node))
z_len = 99
z = z[0:z_len]

for k in range(len(z)):
    z[k].name = "z_{}".format(k)
    z[k].value = 0

q = None
r = None
v_list = None
color_vals = None
p = figure(plot_width=100, plot_height=400, tools="")
p.grid.visible = False
p.axis.visible = False

def init_hex():
    global q, r, v_list, color_vals, p
    q_list = []
    r_list = []
    v_list = []
    color_vals = []
    for k in range(len(z)):
        q_list.append(z[k].q)
        r_list.append(z[k].r)
        v_list.append(z[k].value)
        intense = (3+random.randint(-3,3))/6
        color_vals.append(HSL(intense*40,1,0.5-0.06*hex_distance(z[k])))
        
    q = np.array(q_list)
    r = np.array(r_list)
    cv = np.array(color_vals)
    
    p.background_fill_color  = "black"
    p.height = 256
    p.width = 256
    p.hex_tile(q, r, size=1, fill_color=color_vals,line_color="black", alpha=0.5, hover_color="yellow", name="g")

#init_hex()

x = 0
y = 0
KLD = 0
def change_event(event_string):
    global x, y, z, color_vals
    x,y,_,_ = event_string.split(",")
    x = x.split("=")[1]
    x = float(x)
    y = y.split("=")[1]
    y = float(y)
    x, y = cartesian_to_axial(x, y, 1, "pointytop")
    
    for k in range(len(z)):
        if z[k].q == x and z[k].r == y:
            z[k].value += 1.0
            intense = (3+z[k].value)/6
            color_vals[k]= HSL(intense*40,1,0.5-0.06*hex_distance(z[k]))
    
    push_notebook(handle=bokeh_handle)
    
z = np.random.rand(99,1).astype(np.float64)
p.image(image=[z], x=0, y=0, dw=16,dh=16)
def text_update(update_string):
    # Update a numpy array with rgba values of z[k]
    global p
    p.image(image=z)
    push_notebook(handle=bokeh_handle)
    
    
def callback_event():
    return CustomJS(code="""
    if (IPython.notebook.kernel !== undefined) {
        var kernel = IPython.notebook.kernel;
        cmd = "change_event()";
        kernel.execute(cmd, {}, {});}
    """)
def text_callback():
    return CustomJS(code="""
    var f = cb_obj['value']
    if (IPython.notebook.kernel !== undefined) {
        var kernel = IPython.notebook.kernel;
        cmd = "text_update('"+f+"')";
        kernel.execute(cmd, {}, {});}
    """)

def display_event(attributes=[], style = 'float:left;clear:left;font_size=0.5pt'):
    "Build a suitable CustomJS to display the current event in the div model."
    
    return CustomJS(code="""
        var attrs = %s; var args = [];
        for (var i=0; i<attrs.length; i++ ) {
            args.push(attrs[i] + '=' + Number(cb_obj[attrs[i]]).toFixed(2));
        }
        line = args.join(", ")
        if (IPython.notebook.kernel !== undefined) {
        var kernel = IPython.notebook.kernel;
        cmd = "change_event('"+line+"')";
        kernel.execute(cmd, {}, {});}
        
    """ % (attributes))

point_attributes = ['x','y','sx','sy']  
point_events =  [events.Tap, events.DoubleTap, events.Press]
for event in range(len(point_events)):
    p.js_on_event(point_events[event],display_event(point_attributes))
    
hover = HoverTool(tooltips=None)
p.add_tools(hover)
t = TextInput(callback = text_callback())
tab = Panel(child=row(p,t), title="z sampling")
tabs = Tabs(tabs=[tab])
bokeh_handle = show(tabs, notebook_handle=True)
# To do
# Mouse events for click and double click duration changes z
# after release, z's update rgba image of latent code
# and then you update the example